# 1 Author

**Student Name**: Zhong Zhenghan 

**Student ID**:  210982480

**Github**:  https://github.com/buptxinghan/CBU5201_miniproject_Zhenghan


# 2 Problem formulation
- 微笑检测：确定图片中的人是否在微笑。
- 3D 头部姿态估计：预测头部的3D方向。

# 3 Machine Learning pipeline

- 输入：genki4k 数据集中的一张图片。
- 转换：使用 Haar 级联或 CNN 进行人脸检测和裁剪，数据标准化和增强，使用 CNN 提取特征。
- 模型：微笑检测的分类模型和3D头部姿态的回归模型。
- 输出：微笑/非微笑的二元标签和3D头部姿态标签。

# 4 Transformation stage

- 图像大小调整：所有图像都被调整为统一的大小,数据标准化（像素值缩放）和增强（旋转，翻转等）。
- 使用预训练模型（如 Haar 级联或 CNN）进行面部检测和裁剪。
- 颜色空间转换：如果需要，图像颜色空间从 RGB 转换为灰度。
- 特征工程：提取与微笑和头部姿态相关的特征,使用大型数据集预训练的 CNN（例如 VGG16）提取特征。

# 5 Modelling

- 微笑检测：二元分类模型，例如逻辑回归、SVM 或简单的 CNN。
- 3D 头部姿态估计：回归模型，可以是多层感知器或 CNN。

# 6 Methodology

- 数据划分：训练/验证/测试数据集比例为70%/15%/15%。
- 交叉验证：使用交叉验证来评估模型的泛化能力。
- 性能指标：微笑检测的准确度和混淆矩阵；姿态估计的均方误差。

# 7 Dataset

- 预处理 genki4k 数据集，可视化一些样本。
- 执行任何必要的清理或增强。

# 8 Results

- 展示模型的性能：包括训练过程中的损失和准确度变化图、验证集和测试集上的最终结果。
- 分析结果：讲述模型表现好的地方，以及可能的原因。同时，如果模型某些方面表现不佳，提出可能的原因和解决办法。

# 9 Conclusions

- 概述您的项目和实验结果。
- 讨论项目的成功点和存在的局限。
- 提出改进模型或实验的可能方法。
- 反思整个项目过程，包括学到了什么，遇到了哪些挑战，以及如何克服这些挑战。

在这个项目中，我们成功地实现了一个用于识别人脸微笑的模型。通过使用卷积神经网络，我们在测试集上达到了 XX% 的准确率。尽管结果是令人满意的，但我们发现模型在处理不同光照条件下的图像时存在一定的局限性。未来的工作可以在数据增强和模型鲁棒性方面进行更多的探索。在这个过程中，我们学习了如何处理图像数据、如何选择合适的模型结构以及如何调整模型参数来优化性能。

# 10 Training process

In [26]:
import os
import torch
import torchvision
from torchvision import datasets, models, transforms
from torch.utils.data import DataLoader, random_split
import torch.nn as nn
import torch.optim as optim
from PIL import Image
import cv2
from torch.utils.data import Dataset


1. Pre-Processing

In [27]:
class Genki4kDataset(Dataset):
    def __init__(self, img_dir, labels_file, transform=None):
        self.img_dir = img_dir
        self.transform = transform
        self.labels = []
        with open(labels_file, 'r') as f:
            for line in f:
                parts = line.strip().split()
                smile_label = int(parts[0])  # 第一个值是微笑标签
                self.labels.append(smile_label)
        self.face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        img_path = os.path.join(self.img_dir, f'file{idx+1:04d}.jpg')
        image = cv2.imread(img_path)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

        # 裁剪人脸
        image = self.crop_face(image)

        if self.transform:
            image = Image.fromarray(image)
            image = self.transform(image)

        label = self.labels[idx]
        return image, label

    def crop_face(self, image):
        faces = self.face_cascade.detectMultiScale(image, 1.1, 4)
        for (x, y, w, h) in faces:
            face = image[y:y+h, x:x+w]
            return face  # 返回第一个检测到的人脸
        return image  # 如果没有检测到人脸，返回原图
    
data_transforms = {
    'train': transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'val': transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
}

# data_dir = '/root/MiniProject/MiniProject/genki4k/files'
# labels_path = '/root/MiniProject/MiniProject/genki4k/labels.txt'
data_dir = 'C:/Users/LENOVO/Desktop/MiniProject/MiniProject/genki4k/files'
labels_path = 'C:/Users/LENOVO/Desktop/MiniProject/MiniProject/genki4k/labels.txt'

dataset = Genki4kDataset(data_dir, labels_path, transform=data_transforms['train'])



2. Split the dataset

In [28]:
train_size = int(0.6 * len(dataset))
val_size = int(0.2 * len(dataset))
test_size = len(dataset) - (train_size + val_size)
train_dataset, val_dataset, test_dataset = random_split(dataset, [train_size, val_size, test_size])

train_loader = DataLoader(train_dataset, batch_size=4, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=4, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=4, shuffle=False)

3. 定义模型

In [29]:
# 检查 CUDA 是否可用，定义和移动模型到 GPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

# 将模型移到 GPU
model = models.resnet50(pretrained=True)
num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, 2)  # 2个输出，笑容和不笑
model = model.to(device)

# 定义损失函数和优化器
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)


Using device: cuda


5. 训练模型

In [35]:
def train_model(model, criterion, optimizer, train_loader, val_loader, num_epochs=10):
    for epoch in range(num_epochs):
        model.train()
        running_loss = 0.0
        for i, (inputs, labels) in enumerate(train_loader):
            inputs, labels = inputs.to(device), labels.to(device)

            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()
            if i % 100 == 99:
                print(f'Epoch {epoch+1}, Batch {i+1}, Loss: {running_loss / 100}')
                running_loss = 0.0

        model.eval()
        val_loss = 0.0
        with torch.no_grad():
            for inputs, labels in val_loader:
                inputs, labels = inputs.to(device), labels.to(device)
                outputs = model(inputs)
                loss = criterion(outputs, labels)
                val_loss += loss.item()

        print(f'End of Epoch {epoch+1}, Validation Loss: {val_loss / len(val_loader)}')

    return model

model = train_model(model, criterion, optimizer, train_loader, val_loader, num_epochs=1)

# 保存模型状态字典
torch.save(model.state_dict(), 'model_state_dict.pth')


Epoch 1, Batch 100, Loss: 0.07098883393104188
Epoch 1, Batch 200, Loss: 0.05741179633245338
Epoch 1, Batch 300, Loss: 0.055880295320530425
Epoch 1, Batch 400, Loss: 0.04390445065626409
Epoch 1, Batch 500, Loss: 0.060747919507848566
Epoch 1, Batch 600, Loss: 0.043923204219317995
End of Epoch 1, Validation Loss: 0.24205650706797313


In [ ]:
#加载模型
model = MyModel()  # 替换为实际的模型类名和参数
model.load_state_dict(torch.load('model_state_dict.pth'))


6.   测试和评估模型

In [37]:
def test_model(model, test_loader):
    model.eval()  # 设置模型为评估模式
    correct = 0
    total = 0
    with torch.no_grad():
        for inputs, labels in test_loader:
            inputs, labels = inputs.to(device), labels.to(device)  # 移动数据到 GPU

            outputs = model(inputs)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    print(f'Accuracy of the model on the test images: {100 * correct / total}%')

test_model(model, test_loader)


Accuracy of the model on the test images: 92.125%


7. Analyis the results

In [38]:
from sklearn.metrics import confusion_matrix, classification_report
import numpy as np

def test_model(model, test_loader):
    model.eval()
    all_preds = []
    all_labels = []
    with torch.no_grad():
        for inputs, labels in test_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            _, predicted = torch.max(outputs, 1)
            all_preds.extend(predicted.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())

    return all_preds, all_labels

# 调用测试模型的函数
all_preds, all_labels = test_model(model, test_loader)

# 计算混淆矩阵
conf_matrix = confusion_matrix(all_labels, all_preds)
print("混淆矩阵:")
print(conf_matrix)

# 计算其他性能指标
print("\n分类报告:")
print(classification_report(all_labels, all_preds, target_names=['Not Smiling', 'Smiling']))


混淆矩阵:
[[315  51]
 [  9 425]]

分类报告:
              precision    recall  f1-score   support

 Not Smiling       0.97      0.86      0.91       366
     Smiling       0.89      0.98      0.93       434

    accuracy                           0.93       800
   macro avg       0.93      0.92      0.92       800
weighted avg       0.93      0.93      0.92       800

